# FLEX

This notebook runs [GOTM](https://gotm.net/) using the [FLEX](https://gotm.net/cases/flex/) test case.

Run the case with four turbulence closure schemes:

- GLS-C01A: The generic length scale ([Umlauf and Burchard, 2003](https://doi.org/10.1357/002224003322005087)) model in the $k$-$\epsilon$ formulation with the weak-equilibrium stability function by [Canuto et al., 2001](https://doi.org/10.1175/1520-0485(2001)031%3C1413:OTPIOP%3E2.0.CO;2) (C01A).

- Three variants of KPP via [CVMix](http://cvmix.github.io):
    - KPP-CVMix ([Large et al., 1994](https://doi.org/10.1029/94RG01872), [Griffies et al., 2015](https://github.com/CVMix/CVMix-description/raw/master/cvmix.pdf))
    - KPPLT-VR12 ([Li et al., 2016](https://doi.org/10.1016%2Fj.ocemod.2015.07.020))
    - KPPLT-LF17 ([Li and Fox-Kemper, 2017](https://doi.org/10.1175%2FJPO-D-17-0085.1))
    
Estimate the Stokes drift from the wind following [Li et al., 2017](https://doi.org/10.1016/j.ocemod.2017.03.016).

In [1]:
import sys
import copy
import numpy as np
import matplotlib.pyplot as plt
# add the path of gotmtool
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../../.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
m = Model(name='FLEX', environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/code
   gotmdir_data: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/data
  gotmdir_build: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/build
    gotmdir_exe: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/exe
    gotmdir_run: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/run
 gotmdir_figure: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/figure
   gotmdir_tool: /Users/qingli/work/A2020_CVMix_in_GOTM/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 3.13 ms, sys: 8.76 ms, total: 11.9 ms
Wall time: 93.8 ms


## Configuration
Initialize the GOTM configuration

In [5]:
cfg = m.init_config()

Generating default configuration at '/Users/qingli/work/A2020_CVMix_in_GOTM/gotm/run/FLEX/gotm.yaml'...
Done!


Update the configuration

In [6]:
# setup
title = 'FLEX'
nlev = 145
cfg['title'] = title
cfg['location']['name'] = 'Flex Experiment 1976'
cfg['location']['latitude'] = 58.92
cfg['location']['longitude'] = 0.32
cfg['location']['depth'] = 145.0
cfg['time']['start'] = '1976-04-06 06:00:00'
cfg['time']['stop']  = '1976-06-07 00:00:00'
cfg['time']['dt']    = 360.0
cfg['grid']['nlev']  = nlev
cfg['grid']['ddu']  = 0.0
cfg['grid']['ddl']  = 0.0

# output
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 30

# forcing
datadir = os.getcwd()+'/data'
cfg['temperature']['method'] = 'file'
cfg['temperature']['file'] = datadir+'/t_prof.dat'
cfg['salinity']['method'] = 'file'
cfg['salinity']['file'] = datadir+'/s_prof.dat'
cfg['salinity']['relax']['tau'] = 172800.0
cfg['salinity']['relax']['tau_s'] = 172800.0
cfg['salinity']['relax']['tau_b'] = 172800.0
cfg['surface']['fluxes']['method'] = 'fairall'
cfg['surface']['u10']['method'] = 'file'
cfg['surface']['u10']['file'] = datadir+'/meteo.dat'
cfg['surface']['u10']['column'] = 1
cfg['surface']['v10']['method'] = 'file'
cfg['surface']['v10']['file'] = datadir+'/meteo.dat'
cfg['surface']['v10']['column'] = 2
cfg['surface']['airp']['method'] = 'file'
cfg['surface']['airp']['file'] = datadir+'/meteo.dat'
cfg['surface']['airp']['column'] = 3
cfg['surface']['airp']['scale_factor'] = 100.0
cfg['surface']['airt']['method'] = 'file'
cfg['surface']['airt']['file'] = datadir+'/meteo.dat'
cfg['surface']['airt']['column'] = 4
cfg['surface']['hum']['method'] = 'file'
cfg['surface']['hum']['file'] = datadir+'/meteo.dat'
cfg['surface']['hum']['column'] = 5
cfg['surface']['hum']['type'] = 'wet_bulb'
cfg['surface']['swr']['method'] = 'file'
cfg['surface']['swr']['file'] = datadir+'/swr.dat'
cfg['surface']['longwave_radiation']['method'] = 'file'
cfg['surface']['longwave_radiation']['file'] = datadir+'/lwr.dat'
cfg['surface']['sst']['method'] = 'file'
cfg['surface']['sst']['file'] = datadir+'/sst.dat'

cfg['waves']['stokes_drift']['us']['method'] = 'empirical'
cfg['waves']['stokes_drift']['vs']['method'] = 'empirical'
cfg['waves']['stokes_drift']['empirical']['uwnd']['method'] = 'file'
cfg['waves']['stokes_drift']['empirical']['uwnd']['file'] = datadir+'/meteo.dat'
cfg['waves']['stokes_drift']['empirical']['uwnd']['column'] = 1
cfg['waves']['stokes_drift']['empirical']['vwnd']['method'] = 'file'
cfg['waves']['stokes_drift']['empirical']['vwnd']['file'] = datadir+'/meteo.dat'
cfg['waves']['stokes_drift']['empirical']['vwnd']['column'] = 2

# water type -- Jerlov Type III
cfg['light_extinction']['method'] = 'jerlov-iii'

# tidal
cfg['mimic_3d']['ext_pressure']['type'] = 'elevation'
cfg['mimic_3d']['ext_pressure']['dpdx']['method'] = 'file'
cfg['mimic_3d']['ext_pressure']['dpdx']['file'] = datadir+'/ext_press.dat'
cfg['mimic_3d']['ext_pressure']['dpdx']['column'] = 2
cfg['mimic_3d']['ext_pressure']['dpdy']['method'] = 'file'
cfg['mimic_3d']['ext_pressure']['dpdy']['file'] = datadir+'/ext_press.dat'
cfg['mimic_3d']['ext_pressure']['dpdy']['column'] = 3
cfg['mimic_3d']['ext_pressure']['h']['method'] = 'file'
cfg['mimic_3d']['ext_pressure']['h']['file'] = datadir+'/ext_press.dat'
cfg['mimic_3d']['ext_pressure']['h']['column'] = 1

# EOS 
cfg['eq_state']['method'] = 'unesco'
cfg['eq_state']['form'] = 'full-pot'

# buoyancy
cfg['buoyancy']['NN_ini'] = 1.0e-4


## Run the model

Set the configurations and labels for each run

In [7]:
cfgs = []
labels = []

In [8]:
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'weak_eq_kb_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'canuto-a'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3
cfgs.append(copy.deepcopy(cfg))
labels.append('GLS-C01A_tidal')

In [9]:
# turn tidal forcing off
cfg['mimic_3d']['ext_pressure']['dpdx']['method'] = 'constant'
cfg['mimic_3d']['ext_pressure']['dpdx']['constant_value'] = 0.0
cfg['mimic_3d']['ext_pressure']['dpdy']['method'] = 'constant'
cfg['mimic_3d']['ext_pressure']['dpdy']['constant_value'] = 0.0
cfg['mimic_3d']['ext_pressure']['h']['method'] = 'constant'
cfg['mimic_3d']['ext_pressure']['h']['constant_value'] = 0.0
cfgs.append(copy.deepcopy(cfg))
labels.append('GLS-C01A')

In [10]:
cfg['turbulence']['turb_method'] = 'cvmix'
cfgs.append(copy.deepcopy(cfg))
labels.append('KPP-CVMix')

In [11]:
cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'lwf16'
cfgs.append(copy.deepcopy(cfg))
labels.append('KPPLT-VR12')

In [12]:
cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'lf17'
cfgs.append(copy.deepcopy(cfg))
labels.append('KPPLT-LF17')

Run the cases in parallel with 2 processes 

In [13]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=2)

CPU times: user 17.8 ms, sys: 14.3 ms, total: 32.1 ms
Wall time: 7 s
